In [79]:
%load_ext autoreload
%autoreload 2

import geopandas as gpd
import pandas as pd
import polars as pl
import polars_st as st
import pyogrio

# from electoralyze import region``
from electoralyze.common import constants
from electoralyze.common.geometry import to_geopandas, to_geopolars
from electoralyze.common.testing.region_fixture import (
    RegionMocked,
    create_fake_regions,
    read_true_geometry,
    read_true_metadata,
)
from electoralyze.region.redistribute import redistribute
from electoralyze.region.redistribute.mapping import (
    _create_intersection_area_mapping,
    _get_remaining_area,
    create_region_mapping_base,
)
from polars import testing

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [80]:
intersection_area = _create_intersection_area_mapping(
    geometry_from=read_true_geometry("triangle"),
    geometry_to=read_true_geometry("square"),
)
intersection_area.pipe(print)

shape: (5, 3)
┌──────────┬────────┬─────────┐
│ triangle ┆ square ┆ mapping │
│ ---      ┆ ---    ┆ ---     │
│ str      ┆ str    ┆ f64     │
╞══════════╪════════╪═════════╡
│ A        ┆ main   ┆ 32.0    │
│ B        ┆ main   ┆ 16.0    │
│ C        ┆ main   ┆ 16.0    │
│ B        ┆ null   ┆ 16.0    │
│ C        ┆ null   ┆ 16.0    │
└──────────┴────────┴─────────┘


In [81]:
import shutil
import tempfile

temp_dir = tempfile.TemporaryDirectory(delete=False)
region_ = create_fake_regions(temp_dir.name)
region_
region_.quadrant.geometry
region_.triangle.geometry
region_.rectangle.geometry
region_.square.geometry
region_.l_and_r.geometry
shutil.rmtree(temp_dir.name)

/home/andre/git/private/electoralyze/packages/electoralyze/electoralyze/common/testing/region_fixture.py:260: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  region_gdf.pipe(to_geopandas).to_file(region_shape_file[region_id], driver="ESRI Shapefile")
/home/andre/git/private/electoralyze/.pixi/envs/default/lib/python3.13/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'quadrant_name' to 'quadrant_n'
  ogr_write(
/home/andre/git/private/electoralyze/packages/electoralyze/electoralyze/common/testing/region_fixture.py:260: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  region_gdf.pipe(to_geopandas).to_file(region_shape_file[region_id], driver="ESRI Shapefile")
/home/andre/git/private/electoralyze/.pixi/envs/default/lib/python3.13/site-packages/pyogrio/raw.py:723: RuntimeWarning: Normalized/laundered field name: 'triangle_name' to 'triangle_n'
  ogr

Loading raw...
Extracting...
Transforming...
Saving...
Done!
Loading raw...
Extracting...
Transforming...
Saving...
Done!
Loading raw...
Extracting...
Transforming...
Saving...
Done!
Loading raw...
Extracting...
Transforming...
Saving...
Done!
Loading raw...
Extracting...
Transforming...
Saving...
Done!


In [82]:
from electoralyze.common.testing.region_fixture import ONE_SQUARE_REGION_ID

In [83]:
RegionMocked

electoralyze.common.testing.region_fixture.RegionMocked

In [84]:
data_by_from = pl.DataFrame({"triangle": ["A"], "data": [1]})
print(data_by_from)
redistribute(
    data_by_from,
    region_from=region_.triangle,
    region_to=region_.quadrant,
    mapping="intersection_area",
    redistribute_with_full=False,
)

shape: (1, 2)
┌──────────┬──────┐
│ triangle ┆ data │
│ ---      ┆ ---  │
│ str      ┆ i64  │
╞══════════╪══════╡
│ A        ┆ 1    │
└──────────┴──────┘


quadrant,data
str,f64
"""M""",0.125
"""N""",0.125
"""O""",0.375
"""P""",0.375


In [85]:
data_by_from = pl.DataFrame({"rectangle": ["X", "Y", "Z"], "data": [1, 1, 1]})
print(data_by_from)
redistribute(
    data_by_from,
    region_from=region_.rectangle,
    region_via=region_.quadrant,
    region_to=region_.triangle,
    mapping="intersection_area",
    redistribute_with_full=False,
)

shape: (3, 2)
┌───────────┬──────┐
│ rectangle ┆ data │
│ ---       ┆ ---  │
│ str       ┆ i64  │
╞═══════════╪══════╡
│ X         ┆ 1    │
│ Y         ┆ 1    │
│ Z         ┆ 1    │
└───────────┴──────┘


triangle,data
str,f64
"""B""",0.75
"""A""",1.5
"""C""",0.75


In [86]:
data_by_from = pl.DataFrame({"square": ["main"], "data": [100]})
print(data_by_from)
redistribute(
    data_by_from,
    region_from=region_.square,
    region_via=region_.quadrant,
    region_to=region_.l_and_r,
    mapping="intersection_area",
    redistribute_with_full=False,
)

shape: (1, 2)
┌────────┬──────┐
│ square ┆ data │
│ ---    ┆ ---  │
│ str    ┆ i64  │
╞════════╪══════╡
│ main   ┆ 100  │
└────────┴──────┘


l_and_r,data
str,f64
"""R""",50.0
"""L""",50.0


In [87]:
data_by_from = pl.DataFrame({"square": ["main"], "data": [100]})
print(data_by_from)
redistribute(
    data_by_from,
    region_from=region_.square,
    region_via=region_.triangle,
    region_to=region_.l_and_r,
    mapping="intersection_area",
    redistribute_with_full=False,
)

shape: (1, 2)
┌────────┬──────┐
│ square ┆ data │
│ ---    ┆ ---  │
│ str    ┆ i64  │
╞════════╪══════╡
│ main   ┆ 100  │
└────────┴──────┘


l_and_r,data
str,f64
"""R""",37.5
"""L""",37.5
null,25.0


In [88]:
def test_redistribute_via(
    region: RegionMocked, _name: str, region_ids: dict, redistribute_kwargs: dict, redistributed_expected: dict
):
    redistributed_to = redistribute(
        region_from=region.from_id(region_ids["region_id_from"]),
        region_to=region.from_id(region_ids["region_id_via"]),
        **redistribute_kwargs,
    )

    pl.testing.assert_frame_equal(
        redistributed_to,
        redistributed_expected["to"],
        check_row_order=False,
        check_column_order=False,
    )

    redistributed_via = redistribute(
        region_from=region.from_id(region_ids["region_id_from"]),
        region_via=region.from_id(region_ids["region_id_via"]),
        region_to=region.from_id(region_ids["region_id_to"]),
        **redistribute_kwargs,
    )

    pl.testing.assert_frame_equal(
        redistributed_via,
        redistributed_expected["via"],
        check_row_order=False,
        check_column_order=False,
    )

In [89]:
pl.DataFrame({"l_and_r": ["L", "R"], "data": [32, 64]}).rows(named=True)

[{'l_and_r': 'L', 'data': 32}, {'l_and_r': 'R', 'data': 64}]

In [90]:
data_by_from = pl.DataFrame({"l_and_r": ["L", "R"], "data": [32, 64]})
print(data_by_from)
_name = "L R through Triangle to quadrents"
redistribute_kwargs = dict(
    data_by_from=data_by_from,
    mapping="intersection_area",
    redistribute_with_full=False,
)
region_ids = dict(
    region_id_from="l_and_r",
    region_id_to="quadrant",
    region_id_via="triangle",
)

redistributed_expected = dict(
    via=pl.DataFrame({"quadrant": ["O", "M", "P", None, "N"], "data": [20.0, 12.0, 22.0, 24.0, 18.0]}),
    to=pl.DataFrame({"triangle": ["A", "B", "C"], "data": [48.0, 16.0, 32.0]}),
)

test_redistribute_via(
    _name=_name,
    region=region_,
    region_ids=region_ids,
    redistribute_kwargs=redistribute_kwargs,
    redistributed_expected=redistributed_expected,
)

redistribute(
    region_from=region_.from_id(region_ids["region_id_from"]),
    region_via=region_.from_id(region_ids["region_id_via"]),
    region_to=region_.from_id(region_ids["region_id_to"]),
    **redistribute_kwargs,
)  # .to_dict(as_series=False)
# ).rows(named = True)

shape: (2, 2)
┌─────────┬──────┐
│ l_and_r ┆ data │
│ ---     ┆ ---  │
│ str     ┆ i64  │
╞═════════╪══════╡
│ L       ┆ 32   │
│ R       ┆ 64   │
└─────────┴──────┘


quadrant,data
str,f64
"""P""",22.0
"""N""",18.0
null,24.0
"""M""",12.0
"""O""",20.0


In [91]:
pl.DataFrame(
    {
        ONE_SQUARE_REGION_ID: ["main"],
        "data": [64.0],
    }
).pipe(print)

shape: (1, 2)
┌────────┬──────┐
│ square ┆ data │
│ ---    ┆ ---  │
│ str    ┆ f64  │
╞════════╪══════╡
│ main   ┆ 64.0 │
└────────┴──────┘


In [92]:
redistribute(
    data_by_from=pl.DataFrame(
        {
            ONE_SQUARE_REGION_ID: ["main"],
            "data": [64.0],
        }
    ),
    region_from=region_.square,
    region_via=region_.triangle,
    region_to=region_.quadrant,
    mapping="intersection_area",
    redistribute_with_full=False,
    # aggregation="min",
    errors=False,
).pipe(print)  # .to_dict(as_series=False)
# ).rows(named = True)

shape: (5, 2)
┌──────────┬──────┐
│ quadrant ┆ data │
│ ---      ┆ ---  │
│ str      ┆ f64  │
╞══════════╪══════╡
│ null     ┆ 16.0 │
│ P        ┆ 14.0 │
│ M        ┆ 10.0 │
│ O        ┆ 14.0 │
│ N        ┆ 10.0 │
└──────────┴──────┘


In [93]:
# region_.quadrant.geometry.pipe(to_geopandas).pipe(print)
# region_.square.geometry.pipe(to_geopandas).pipe(print)
# region_.triangle.geometry.pipe(to_geopandas).pipe(print)
# region_.quadrant.geometry.pipe(print)
# region_.square.geometry.pipe(print)
# region_.triangle.geometry.pipe(print)

In [94]:
redistribute_kwargs["data_by_from"].pipe(print)

shape: (2, 2)
┌─────────┬──────┐
│ l_and_r ┆ data │
│ ---     ┆ ---  │
│ str     ┆ i64  │
╞═════════╪══════╡
│ L       ┆ 32   │
│ R       ┆ 64   │
└─────────┴──────┘


In [95]:
redistributed_expected.get("to").rows(named=True)

[{'triangle': 'A', 'data': 48.0},
 {'triangle': 'B', 'data': 16.0},
 {'triangle': 'C', 'data': 32.0}]

In [96]:
from electoralyze.common.testing.region_fixture import FOUR_SQUARE_REGION_ID, LEFT_RIGHT_REGION_ID, ONE_SQUARE_REGION_ID

In [97]:
_name = "from one square -> via quadrants -> to L and R, "
region_ids = dict(
    region_id_from=ONE_SQUARE_REGION_ID,
    region_id_via=FOUR_SQUARE_REGION_ID,
    region_id_to=LEFT_RIGHT_REGION_ID,
)
redistribute_kwargs = dict(
    data_by_from=pl.DataFrame(
        [
            {ONE_SQUARE_REGION_ID: "O", "data": 100},
        ]
    ),
    mapping="intersection_area",
    redistribute_with_full=True,
)
redistributed_expected = dict(
    to=pl.DataFrame(
        [
            {FOUR_SQUARE_REGION_ID: "N", "data": 25.0},
            {FOUR_SQUARE_REGION_ID: "M", "data": 25.0},
            {FOUR_SQUARE_REGION_ID: "O", "data": 25.0},
            {FOUR_SQUARE_REGION_ID: "P", "data": 25.0},
        ]
    ),
    via=pl.DataFrame(
        [
            {LEFT_RIGHT_REGION_ID: "L", "data": 50.0},
            {LEFT_RIGHT_REGION_ID: "R", "data": 50.0},
        ]
    ),
)

In [98]:
_create_intersection_area_mapping(
    region_.from_id(region_ids["region_id_from"]).geometry,
    region_.from_id(region_ids["region_id_via"]).geometry,
).pipe(print)

shape: (4, 3)
┌────────┬──────────┬─────────┐
│ square ┆ quadrant ┆ mapping │
│ ---    ┆ ---      ┆ ---     │
│ str    ┆ str      ┆ f64     │
╞════════╪══════════╪═════════╡
│ main   ┆ M        ┆ 16.0    │
│ main   ┆ N        ┆ 16.0    │
│ main   ┆ O        ┆ 16.0    │
│ main   ┆ P        ┆ 16.0    │
└────────┴──────────┴─────────┘


In [99]:
# region_.from_id(region_ids["region_id_from"]).geometry.pipe(to_geopandas).explore()

In [100]:
# region_.from_id(region_ids["region_id_via"]).geometry.pipe(to_geopandas).explore()

In [101]:
redistribute(
    region_from=region_.from_id(region_ids["region_id_from"]),
    # region_via=region_.from_id(region_via_id),
    region_to=region_.from_id(region_ids["region_id_via"]),
    # region_to=region_.from_id(region_to_id),
    data_by_from=redistribute_kwargs["data_by_from"],
    mapping=redistribute_kwargs["mapping"],
    redistribute_with_full=False,
    errors="warning",
)

Miss match in data for column: 'data'. From: 100 -> To: 0.0


quadrant,data
str,f64


In [102]:
from polars.exceptions import ColumnNotFoundError

try:
    region_.from_id(ONE_SQUARE_REGION_ID).geometry.select("square1")
except ColumnNotFoundError:
    print("das")

das


In [103]:
redistribute(
    region_from=region_.from_id(ONE_SQUARE_REGION_ID),
    region_to=region_.from_id(ONE_SQUARE_REGION_ID),
    data_by_from=pl.DataFrame({ONE_SQUARE_REGION_ID: ["O"], "data": [100]}),
    # create_region_mapping_base=False,
    redistribute_with_full=False,
)

ValueError: `from` and `to` region cannot be the same. Both were 'square'

In [104]:
test_redistribute_via(
    _name=_name,
    region=region_,
    region_ids=region_ids,
    redistribute_kwargs=redistribute_kwargs,
    redistributed_expected=redistributed_expected,
)

Extracting...


DataSourceError: /tmp/tmp9dg8igyy/raw_geometry/square/shape.shp: No such file or directory